In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df = df.dropna()
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [5]:
X = df.drop('label', axis=1)
Y = df['label']

In [6]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [7]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [8]:
X.shape, Y.shape

((18285, 4), (18285,))

In [9]:
import tensorflow as tf
tf.__version__

'2.17.1'

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense

In [11]:
voc_size = 5000

In [12]:
messages = X.copy()

In [13]:
messages.reset_index(inplace=True)

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [19]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4350, 2190, 3894],
       [   0,    0,    0, ...,  417,  229, 2503],
       [   0,    0,    0, ...,  964, 3429, 3166],
       ...,
       [   0,    0,    0, ..., 2477, 2135, 4672],
       [   0,    0,    0, ..., 2245, 1138, 2120],
       [   0,    0,    0, ..., 2599, 3128, 1473]], dtype=int32)

In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4325,
       3538,  828,  216, 4220, 1898, 3833, 4350, 2190, 3894], dtype=int32)

In [21]:
embedding_vector_features = 40  # each word is converted to vector of size 40 (inherit algo - eg:CBOW)
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
len(embedded_docs), Y.shape

(18285, (18285,))

In [24]:
import numpy as np

X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [25]:
X_final.shape, Y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.2, random_state=42)

In [27]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7987 - loss: 0.4083 - val_accuracy: 0.9089 - val_loss: 0.2063
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9447 - loss: 0.1372 - val_accuracy: 0.9130 - val_loss: 0.2216
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9628 - loss: 0.0990 - val_accuracy: 0.9021 - val_loss: 0.2240
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9724 - loss: 0.0803 - val_accuracy: 0.9117 - val_loss: 0.2282
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9839 - loss: 0.0522 - val_accuracy: 0.9098 - val_loss: 0.2946
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9892 - loss: 0.0325 - val_accuracy: 0.9130 - val_loss: 0.3311
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9951 - loss: 0.0171 - val_accuracy: 0.9098 - val_loss: 0.3450
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9972 - loss: 0.0126 - val_accuracy: 0.

In [28]:
y_pred = model.predict(X_test)

115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [29]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [32]:
confusion_matrix(Y_test, y_pred)

array([[1874,  208],
       [ 118, 1457]])

In [35]:
accuracy_score(Y_test, y_pred)

0.9108558928083128

In [38]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      2082
           1       0.88      0.93      0.90      1575

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657

